<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.3 Data Access and Basic Processing

<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 10px; max-height: 80px;"/>

## Filter Operators with Pangeo ecosystem

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators.

Let's start again with the same sample data from the Sentinel-2 STAC Collection with an additional filter.

### Properties Filter

When working with optical data like Sentinel-2, most of the times we would like to discard cloudy acquisitions as soon as possible.

We can do it using a property filter: in this case we want to keep only the acquisitions with less than 50% cloud coverage.

In [ ]:
import pystac_client
import stackstac
import numpy as np
from pyproj import Proj

In [ ]:
properties = {"eo:cloud_cover": dict(lt=50)}

In [ ]:
spatial_extent = [11.1, 46.1, 11.5, 46.5]
temporal_extent = ["2024-01-01T00:00:00Z","2024-10-30T00:00:00Z"]
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
items = catalog.search(
    bbox=spatial_extent,
    datetime=temporal_extent,
    collections=["sentinel-2-l2a"],
    query=properties
).item_collection()

datacube = stackstac.stack(items,
                     bounds_latlon=spatial_extent,
)
datacube

### Temporal Filter

To filter along time the data collection with Pangeo Xarray, we create a binary mask combining two conditions, passing it to the `where` method from Xarray. Remember to set `drop=True` to discard the dates outside the selected period.

**Note**: We cannot use the sel method to select a slice to filter on the time dimension since the times are not equally spaced.

In [ ]:
temporal_slice = datacube.where((datacube.time>=np.datetime64("2024-05-10T00:00:00")) &
                                (datacube.time<=np.datetime64("2024-06-30T00:00:00")),
                                drop=True)
temporal_slice

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

**Quiz hint: look carefully at the dimensions of the resulting datacube!**

### Spatial Filter

To slice along the spatial dimensions the data collection with Pangeo, we can use `sel` method with a `slice`.

Let's get the spatial extent expressed in the Coordinate Reference System of the datacube. We need to project latitudes, longitudes to the data cube CRS.

In [ ]:
west = 11.259613; east = 11.406212
south = 46.461019; north = 46.522237
projection = Proj(temporal_slice.attrs["crs"])
x_ws, y_ws = projection(west, south)
x_wn, y_wn = projection(west, north)
x_es, y_es = projection(east, south)
x_en, y_en = projection(east, north)
xmax = max(x_ws, x_wn, x_es, x_en)
xmin = min(x_ws, x_wn, x_es, x_en)
ymax = max(y_ws, y_wn, y_es, y_en)
ymin = min(y_ws, y_wn, y_es, y_en)
print(xmin, xmax, ymin, ymax)

The `sel` method with `slice` is used with a set of coordinates. 

**Note**: The order of the interval in the slice needs to be expressed in the same order as the corresponding coordinate.

In [ ]:
spatial_slice = temporal_slice.sel(x=slice(xmin,xmax), y = slice(ymax,ymin))
spatial_slice

**Quiz hint: look carefully at the dimensions of the loaded datacube!**

### Bands Filter

To slice along the bands dimension, keeping only the necessary bands, we can use the `sel` method and `isin`.

In [ ]:
bands = ["red", "green", "blue"]
bands_slice = spatial_slice.sel(band=spatial_slice.band.isin(bands))
bands_slice